In [17]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime

In [18]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [19]:
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker,start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-20  155.070007  157.820007  154.149994  157.399994  157.182587   
       2023-03-21  157.320007  159.399994  156.539993  159.279999  159.059982   
       2023-03-22  159.300003  162.139999  157.809998  157.830002  157.612000   
       2023-03-23  158.830002  161.550003  157.679993  158.929993  158.710464   
       2023-03-24  158.860001  160.339996  157.850006  160.250000  160.028656   

                     Volume  
Ticker Date                  
AAPL   2023-03-20  73641400  
       2023-03-21  73938300  
       2023-03-22  75701800  
      

In [20]:
df.head()

Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-20  155.070007  157.820007  154.149994  157.399994  157.182587   
       2023-03-21  157.320007  159.399994  156.539993  159.279999  159.059982   
       2023-03-22  159.300003  162.139999  157.809998  157.830002  157.612000   
       2023-03-23  158.830002  161.550003  157.679993  158.929993  158.710464   
       2023-03-24  158.860001  160.339996  157.850006  160.250000  160.028656   

                     Volume  
Ticker Date                  
AAPL   2023-03-20  73641400  
       2023-03-21  73938300  
       2023-03-22  75701800  
       2023-03-23  67622100  
       2023-03-24  59196500

In [21]:
df = df.reset_index()
df.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-03-20,155.070007,157.820007,154.149994,157.399994,157.182587,73641400
1,AAPL,2023-03-21,157.320007,159.399994,156.539993,159.279999,159.059982,73938300
2,AAPL,2023-03-22,159.300003,162.139999,157.809998,157.830002,157.612000,75701800
3,AAPL,2023-03-23,158.830002,161.550003,157.679993,158.929993,158.710464,67622100
4,AAPL,2023-03-24,158.860001,160.339996,157.850006,160.250000,160.028656,59196500


In [22]:
import plotly.express as px
fig = px.line(df, x='Date', y='Close', color='Ticker', title='Stock Market Performance for The Last 3 Months')
fig.show()

In [23]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
             facet_col='Ticker',
             labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
             title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [24]:
df['MA 10d'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA 20d'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker,group in df.groupby('Ticker'):
    print(f'Moving Average for {ticker}')
    print(group[['MA 10d', 'MA 20d']])

Moving Average for AAPL
        MA 10d      MA 20d
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  179.752002  176.529002
59  180.353001  177.091001
60  181.023001  177.685001
61  181.615001  178.351000
62  182.012001  178.844500

[63 rows x 2 columns]
Moving Average for GOOG
         MA 10d      MA 20d
189         NaN         NaN
190         NaN         NaN
191         NaN         NaN
192         NaN         NaN
193         NaN         NaN
..          ...         ...
247  124.498001  123.543000
248  124.477001  123.916500
249  124.578001  124.131001
250  124.720000  124.346500
251  124.603000  124.373500

[63 rows x 2 columns]
Moving Average for MSFT
         MA 10d      MA 20d
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
..          ...         ...
121  330.448001  324.28200

In [25]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA 10d', 'MA 20d'],
                 title=f'{ticker} Moving Averages')
    fig.show()

In [26]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [27]:
#create a Dataframe with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']]
apple = apple.rename(columns={'Close': 'AAPL'})

microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']]
microsoft = microsoft.rename(columns={'Close': 'MSFT'})

df_corr = pd.merge(apple, microsoft, on='Date')

#create a scaatter plot to visualize the correlation

fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                trendline='ols',
                title='Correlation between Apple and Microsoft')
fig.show()